In [37]:
# Installing required libraries: Hugging Face Transformers, Datasets, and Scikit-learn for model building, dataset handling, and evaluation
!pip install transformers datasets scikit-learn --quiet

In [38]:
# Importing all the necessary libraries
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [39]:
# Loading the dataset
df = pd.read_csv("/content/drive/MyDrive/Fact-checker-ML-dataset/politifact_factcheck_data.csv")

# Displaying the first 5 rows
df.head()

,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
0,true,Barack Obama,John McCain opposed bankruptcy protections for...,6/11/2008,speech,Adriel Bettelheim,6/16/2008,https://www.politifact.com/factchecks/2008/jun...
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",6/7/2022,television,Yacob Reyes,6/13/2022,https://www.politifact.com/factchecks/2022/jun...
2,mostly-true,Kelly Ayotte,"Says Maggie Hassan was ""out of state on 30 day...",5/18/2016,news,Clay Wirestone,5/27/2016,https://www.politifact.com/factchecks/2016/may...
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",2/1/2021,blog,Madison Czopek,2/5/2021,https://www.politifact.com/factchecks/2021/feb...
4,half-true,Bobby Jindal,"""I'm the only (Republican) candidate that has ...",8/30/2015,television,Linda Qiu,8/30/2015,https://www.politifact.com/factchecks/2015/aug...


In [40]:
# Displays all the column names and the data types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21152 entries, 0 to 21151
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   verdict                  21152 non-null  object
 1   statement_originator     21152 non-null  object
 2   statement                21152 non-null  object
 3   statement_date           21152 non-null  object
 4   statement_source         21152 non-null  object
 5   factchecker              21131 non-null  object
 6   factcheck_date           21152 non-null  object
 7   factcheck_analysis_link  21152 non-null  object
dtypes: object(8)
memory usage: 1.3+ MB


In [41]:
# Normalizing the verdicts
df["verdict"] = df["verdict"].str.strip().str.lower()

In [42]:
# Finding the unique values of "verdict"
print(df['verdict'].unique())

['true' 'false' 'mostly-true' 'half-true' 'pants-fire' 'mostly-false']


In [43]:
# Drop missing values
df = df.dropna(subset=["statement", "verdict", "factchecker"])

# Map verdicts to binary values
binary_labels = {
    "true": "true",
    "mostly-true": "true",
    "half-true": "true",
    "mostly-false": "false",
    "false": "false",
    "pants-fire": "false"
}

# Filtering and Mapping
df = df[df["verdict"].isin(binary_labels.keys())].copy()
df["label"] = df["verdict"].map(binary_labels)

In [44]:
# Droping unused columns
df_t5 = df[["statement", "label"]].dropna()

# Formating for T5 model
df["input_text"] = "statement: " + df["statement_originator"].fillna("") + " says: " + df["statement"].str.strip()
df["target_text"] = df["label"]

In [45]:
# Displays all the column names and the data types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21131 entries, 0 to 21151
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   verdict                  21131 non-null  object
 1   statement_originator     21131 non-null  object
 2   statement                21131 non-null  object
 3   statement_date           21131 non-null  object
 4   statement_source         21131 non-null  object
 5   factchecker              21131 non-null  object
 6   factcheck_date           21131 non-null  object
 7   factcheck_analysis_link  21131 non-null  object
 8   label                    21131 non-null  object
 9   input_text               21131 non-null  object
 10  target_text              21131 non-null  object
dtypes: object(11)
memory usage: 1.9+ MB


In [46]:
# Optional: Balance classes if imbalanced #OPTIONAL
from sklearn.utils import resample

label_counts = df["label"].value_counts()
print("Label Distribution Before Balancing:\n", label_counts)

min_class = label_counts.min()
df_balanced = df.groupby("label").sample(n=min_class, random_state=42).reset_index(drop=True)

print("Label Distribution After Balancing:\n", df_balanced['label'].value_counts())

Label Distribution Before Balancing:
 label
false    11756
true      9375
Name: count, dtype: int64
Label Distribution After Balancing:
 label
false    9375
true     9375
Name: count, dtype: int64


In [47]:
# Step 4: Stratified train-test split
# from sklearn.model_selection import train_test_split

# train_df, test_df = train_test_split(
#     df_balanced,
#     test_size=0.2,
#     random_state=42,
#     stratify=df_balanced["label"]
# )

# # Reset index
# train_df = train_df.reset_index(drop=True)
# test_df = test_df.reset_index(drop=True)

# print("Train size:", len(train_df))
# print("Test size:", len(test_df))

In [48]:
# Checking distribution
print(df['target_text'].value_counts())

target_text
false    11756
true      9375
Name: count, dtype: int64


In [49]:
# Compute sample weights to handle the class imbalance during model training
class_weights = class_weight.compute_sample_weight('balanced', df['target_text'])

In [50]:
# Split into initial train (80%) and temporary (20%)
train_df, temp_df = train_test_split(
    df_balanced,
    test_size=0.2,
    stratify=df_balanced["target_text"],
    random_state=42
)

# Split temporary into validation (10%) and test (10%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["target_text"],
    random_state=42
)

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

Train size: 15000
Validation size: 1875
Test size: 1875


In [51]:
print(train_df[["input_text", "target_text"]].head())

                                              input_text target_text
2068   statement: Alan Grayson says: A bill to advanc...       false
11852  statement: Democratic Governors Association sa...        true
14014  statement: Mark Warner says: Says the Trump ad...        true
3017   statement: Adam Smith says: "Go back to 2005 u...       false
2222   statement: Facebook posts says: Congress creat...       false


In [52]:
# Base directory path in Google Drive
base_save_path = "/content/drive/MyDrive/test-train-val-new-fact-cheker-ml-datasets/"

# Create separate directories for each dataset
train_dir = os.path.join(base_save_path, "train")
val_dir = os.path.join(base_save_path, "validation")
test_dir = os.path.join(base_save_path, "test")

# Ensure directories exist (create if not)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Save datasets in their respective directories
train_df[["input_text", "target_text"]].to_csv(os.path.join(train_dir, "train_data.csv"), index=False)
val_df[["input_text", "target_text"]].to_csv(os.path.join(val_dir, "validation_data.csv"), index=False)
test_df[["input_text", "target_text"]].to_csv(os.path.join(test_dir, "test_data.csv"), index=False)

print("✅ Datasets saved in separate directories:")
print(f"Train: {train_dir}/train_data.csv")
print(f"Validation: {val_dir}/validation_data.csv")
print(f"Test: {test_dir}/test_data.csv")

✅ Datasets saved in separate directories:
Train: /content/drive/MyDrive/test-train-val-new-fact-cheker-ml-datasets/train/train_data.csv
Validation: /content/drive/MyDrive/test-train-val-new-fact-cheker-ml-datasets/validation/validation_data.csv
Test: /content/drive/MyDrive/test-train-val-new-fact-cheker-ml-datasets/test/test_data.csv
